In [8]:
import neo4j
import csv
import pandas as pd

driver = neo4j.GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "1234")) 
driver.verify_connectivity()

In [9]:
def run_query(driver, query):
    with driver.session() as session:
        # result = session.run(query)
        result = session.run(query)
        if result != None: 
            return result.value()
        else:
            return None

def clear_relations(driver):
        delete_relation_query = f"""
            MATCH ()-[r]-()
            CALL {{
                WITH r
                DELETE r
            }} IN TRANSACTIONS
            """
        run_query(driver, delete_relation_query)

def clear_nodes(driver):
    delete_node_query = f"""
        MATCH (n)
        CALL {{
            WITH n
            DELETE n
        }} IN TRANSACTIONS
        """
    run_query(driver, delete_node_query)

def clear_db(driver):
    clear_relations(driver)
    clear_nodes(driver)

def connect_event_application(driver):
    with driver.session() as session:
        session.run(f"""
        MATCH (e:Event)
        MATCH (a:Application) WHERE e.application = a.application
        CALL {{
            WITH a, e
            MERGE (e)-[:LOG]->(a)
        }} IN TRANSACTIONS
        """)
        
def create_entity_nodes(driver):
    entity_types = ["doctype", "resource", "subprocess"]
    with driver.session() as session:
        for entity_type in entity_types:
            session.run(f"""
                MATCH (e:Event) 
                WITH e UNWIND e.{entity_type} AS id_val
                WITH DISTINCT id_val
                CALL {{
                    WITH id_val
                    MERGE (:Entity {{ID:id_val, EntityType:"{entity_type}"}})
                }} IN TRANSACTIONS
                """)
            session.run(f"""
                MATCH (e:Event) 
                MATCH (n:Entity {{EntityType: "{entity_type}"}}) WHERE e.{entity_type} = n.ID
                CALL {{
                    WITH e, n
                    MERGE (e)-[:CORR]->(n)
                }} IN TRANSACTIONS
                """)

def create_df_logs(driver):
    with driver.session() as session:
        session.run(f"""
            MATCH (a:Application)
            CALL {{
                WITH a
                MATCH (a)<-[:LOG]-(e)
                WITH a, e AS events ORDER BY e.timestamp
                WITH a, collect(events) AS event_node_list
                UNWIND range(0, size(event_node_list) - 2) as i
                WITH a, event_node_list[i] AS e1, event_node_list[i + 1] AS e2
                MERGE (e1)-[df:DF_LOG {{Application:a.application}}]->(e2)
            }} IN TRANSACTIONS
            """)
        
def create_df_entities(driver):
    with driver.session() as session:
        session.run(f"""
            MATCH (n:Entity)
            CALL {{
                WITH n
                MATCH (n)<-[:CORR]-(e)
                WITH n, e AS events ORDER BY e.timestamp
                WITH n, collect(events) AS event_node_list
                UNWIND range(0, size(event_node_list) - 2) as i
                WITH n, event_node_list[i] AS e1, event_node_list[i + 1] AS e2
                MERGE (e1)-[df:DF_ENTITY {{EntityType:n.EntityType, ID:n.ID}}]->(e2)
            }} IN TRANSACTIONS
            """)


In [3]:
# clear_db(driver)

In [10]:
connect_event_application(driver)

In [11]:
create_df_logs(driver)

In [12]:
create_entity_nodes(driver)

In [13]:
create_df_entities(driver)